In [1]:
import pandas as pd
import numpy as np
import os
import math

INPUT_DIR = 'data/processed_landmarks'
FEATURES_DIR = 'data/features'
os.makedirs(FEATURES_DIR, exist_ok=True)

# Cargar un archivo de prueba
file = os.listdir(INPUT_DIR)[0]
df = pd.read_csv(os.path.join(INPUT_DIR, file))
print(f"Cargando: {file}")
df.head()


Cargando: 20250520_162240.csv


,frame,x0,y0,z0,v0,x1,y1,z1,v1,x2,...,z30,v30,x31,y31,z31,v31,x32,y32,z32,v32
0,1,-0.566942,0.045531,0.157942,0.999290,-0.588868,0.070639,0.087626,0.999503,-0.587042,...,0.680871,0.856221,0.401207,0.108973,0.705407,0.888463,0.423777,-0.109207,0.684138,0.890916
1,2,-0.574976,0.048363,0.119633,0.998929,-0.597243,0.074421,0.050482,0.999298,-0.595474,...,0.632803,0.821580,0.415066,0.129866,0.715153,0.817014,0.477795,-0.134838,0.607095,0.814448
2,3,-0.581042,0.051428,0.094034,0.998634,-0.603550,0.078090,0.026161,0.999099,-0.601852,...,0.594707,0.797935,0.426085,0.142664,0.716119,0.754302,0.515185,-0.156124,0.549707,0.751401
3,4,-0.585141,0.054726,0.081147,0.998338,-0.607788,0.081646,0.014663,0.998891,-0.606176,...,0.566581,0.770815,0.434264,0.147369,0.708304,0.690598,0.535950,-0.173064,0.511973,0.690454
4,5,-0.586219,0.057776,0.081613,0.997993,-0.608941,0.084342,0.017437,0.998633,-0.607426,...,0.554049,0.745630,0.440998,0.141150,0.676118,0.640223,0.524803,-0.179074,0.506909,0.634820


In [2]:
def calcular_angulo(a, b, c):
    """Calcula el ángulo en el punto B formado por los puntos A-B-C."""
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))  # Evitar errores numéricos
    return np.degrees(angle)


In [3]:
def extraer_features_por_frame(frame):
    features = {}
    coords = {}

    # Cargar coordenadas en un diccionario {0: np.array([x,y,z]), ..., 32: ...}
    for i in range(33):
        coords[i] = np.array([frame[f'x{i}'], frame[f'y{i}'], frame[f'z{i}']])

    # Ángulos articulares (puedes agregar más)
    features['angle_elbow_left'] = calcular_angulo(coords[11], coords[13], coords[15])
    features['angle_elbow_right'] = calcular_angulo(coords[12], coords[14], coords[16])
    features['angle_knee_left'] = calcular_angulo(coords[23], coords[25], coords[27])
    features['angle_knee_right'] = calcular_angulo(coords[24], coords[26], coords[28])
    features['angle_shoulder_left'] = calcular_angulo(coords[13], coords[11], coords[23])
    features['angle_shoulder_right'] = calcular_angulo(coords[14], coords[12], coords[24])
    
    # Inclinación lateral del torso (diferencia vertical entre hombros)
    features['torso_inclination'] = coords[11][1] - coords[12][1]

    return features


In [ ]:
df_features = df.apply(extraer_features_por_frame, axis=1, result_type='expand')
df_features.insert(0, 'frame', df['frame'])
df_features.head()


In [ ]:
output_path = os.path.join(FEATURES_DIR, file)
df_features.to_csv(output_path, index=False)
print(f"✅ Guardado: {output_path}")
